# Imports

In [1]:
import math
import os
import numpy as np
import tensorflow as tf

from neuraxle.api.flask import FlaskRestApiWrapper
from neuraxle.base import ExecutionContext, DEFAULT_CACHE_FOLDER, ExecutionMode, BaseStep
from neuraxle.hyperparams.space import HyperparameterSamples
from neuraxle.steps.encoding import OneHotEncoder
from neuraxle.pipeline import MiniBatchSequentialPipeline, Joiner
from neuraxle.steps.output_handlers import OutputTransformerWrapper

# TODO: move in a package neuraxle-tensorflow 
from savers.tensorflow1_step_saver import TensorflowV1StepSaver
from steps.custom_json_decoder_for_2darray import CustomJSONDecoderFor2DArray
from steps.custom_json_encoder_of_outputs import CustomJSONEncoderOfOutputs
from data_reading import DATASET_PATH, TRAIN, TEST, X_train_signals_paths, load_X, load_y, \
    TRAIN_FILE_NAME, TEST_FILE_NAME
from savers.tensorflow1_step_saver import TensorflowV1StepSaver
from pipeline import HumanActivityRecognitionPipeline, BATCH_SIZE


# Download Data

In [2]:
# Note: Linux bash commands start with a "!" inside those "ipython notebook" cells

DATA_PATH = "data/"

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

!python download_dataset.py

!pwd && ls
os.chdir("..")
!pwd && ls

DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
print("\n" + "Dataset is now located at: " + DATASET_PATH)

/home/gui/Documents/GIT/refactor_lstm/LSTM-Human-Activity-Recognition
1_train_and_save_LSTM.ipynb	data_reading.py  old.py		   savers
2_call_rest_api_and_eval.ipynb	LICENSE		 pipeline.py	   steps
Call-API.ipynb			LSTM_files	 __pycache__	   venv
call_api.py			LSTM_new.ipynb	 README.md
data				new.py		 requirements.txt
/home/gui/Documents/GIT/refactor_lstm/LSTM-Human-Activity-Recognition/data
 download_dataset.py   source.txt	 'UCI HAR Dataset.zip'
 __MACOSX	      'UCI HAR Dataset'

Downloading...
Dataset already downloaded. Did not download twice.

Extracting...
Dataset already extracted. Did not extract twice.

/home/gui/Documents/GIT/refactor_lstm/LSTM-Human-Activity-Recognition/data
 download_dataset.py   source.txt	 'UCI HAR Dataset.zip'
 __MACOSX	      'UCI HAR Dataset'
/home/gui/Documents/GIT/refactor_lstm/LSTM-Human-Activity-Recognition
1_train_and_save_LSTM.ipynb	data_reading.py  old.py		   savers
2_call_rest_api_and_eval.ipynb	LICENSE		 pipeline.py	   steps
Call-API.ipynb			LSTM

# Load data

In [3]:
# Load "X" (the neural network's training and testing inputs)

X_train = load_X(X_train_signals_paths)
# X_test = load_X(X_test_signals_paths)

# Load "y" (the neural network's training and testing outputs)

y_train_path = os.path.join(DATASET_PATH, TRAIN, TRAIN_FILE_NAME)
# y_test_path = os.path.join(DATASET_PATH, TEST, TEST_FILE_NAME)

y_train = load_y(y_train_path)
# y_test = load_y(y_test_path)

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_train.shape, y_train.shape, np.mean(X_train), np.std(X_train))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(7352, 128, 9) (7352, 1) 0.10206611 0.40216514
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


# LSTM RNN Model Forward

In [4]:
def tf_model_forward(pred_name, name_x, name_y, hyperparams):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters.
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network.
    # Note, some code of this notebook is inspired from an slightly different
    # RNN architecture used on another dataset, some of the credits goes to
    # "aymericdamien" under the MIT license.
    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)

    # Graph input/output
    x = tf.placeholder(tf.float32, [None, hyperparams['n_steps'], hyperparams['n_inputs']], name=name_x)
    y = tf.placeholder(tf.float32, [None, hyperparams['n_classes']], name=name_y)

    # Graph weights
    weights = {
        'hidden': tf.Variable(
            tf.random_normal([hyperparams['n_inputs'], hyperparams['n_hidden']])
        ),  # Hidden layer weights
        'out': tf.Variable(
            tf.random_normal([hyperparams['n_hidden'], hyperparams['n_classes']], mean=1.0)
        )
    }

    biases = {
        'hidden': tf.Variable(
            tf.random_normal([hyperparams['n_hidden']])
        ),
        'out': tf.Variable(
            tf.random_normal([hyperparams['n_classes']])
        )
    }

    data_inputs = tf.transpose(
        x,
        [1, 0, 2])  # permute n_steps and batch_size

    # Reshape to prepare input to hidden activation
    data_inputs = tf.reshape(data_inputs, [-1, hyperparams['n_inputs']])
    # new shape: (n_steps*batch_size, n_input)

    # ReLU activation, thanks to Yu Zhao for adding this improvement here:
    _X = tf.nn.relu(
        tf.matmul(data_inputs, weights['hidden']) + biases['hidden']
    )

    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, hyperparams['n_steps'], 0)
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(hyperparams['n_hidden'], forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(hyperparams['n_hidden'], forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)

    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many-to-one" style classifier,
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]

    # Linear activation
    pred = tf.matmul(lstm_last_output, weights['out']) + biases['out']
    return tf.identity(pred, name=pred_name)

# Neuraxle RNN TensorFlow Model Step

In [5]:
LSTM_RNN_VARIABLE_SCOPE = "lstm_rnn"
X_NAME = 'x'
Y_NAME = 'y'
PRED_NAME = 'pred'

N_HIDDEN = 32
N_STEPS = 128
N_INPUTS = 9
LAMBDA_LOSS_AMOUNT = 0.0015
LEARNING_RATE = 0.0025
N_CLASSES = 6
BATCH_SIZE = 1500

class ClassificationRNNTensorFlowModel(BaseStep):
    HYPERPARAMS = HyperparameterSamples({
        'n_steps': N_STEPS,  # 128 timesteps per series
        'n_inputs': N_INPUTS,  # 9 input parameters per timestep
        'n_hidden': N_HIDDEN,  # Hidden layer num of features
        'n_classes': N_CLASSES,  # Total classes (should go up, or should go down)
        'learning_rate': LEARNING_RATE,
        'lambda_loss_amount': LAMBDA_LOSS_AMOUNT,
        'batch_size': BATCH_SIZE
    })

    def __init__(
            self
    ):
        BaseStep.__init__(
            self,
            hyperparams=ClassificationRNNTensorFlowModel.HYPERPARAMS,
            savers=[TensorflowV1StepSaver()]
        )

        self.graph = None
        self.sess = None
        self.l2 = None
        self.cost = None
        self.optimizer = None
        self.correct_pred = None
        self.accuracy = None
        self.test_losses = None
        self.test_accuracies = None
        self.train_losses = None
        self.train_accuracies = None

    def strip(self):
        self.sess = None
        self.graph = None
        self.l2 = None
        self.cost = None
        self.optimizer = None
        self.correct_pred = None
        self.accuracy = None

    def setup(self) -> BaseStep:
        if self.is_initialized:
            return self

        self.create_graph()

        with self.graph.as_default():
            # Launch the graph
            with tf.variable_scope(LSTM_RNN_VARIABLE_SCOPE, reuse=tf.AUTO_REUSE):

                pred = tf_model_forward(PRED_NAME, X_NAME, Y_NAME, self.hyperparams)

                # Loss, optimizer and evaluation
                # L2 loss prevents this overkill neural network to overfit the data

                l2 = self.hyperparams['lambda_loss_amount'] * sum(
                    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
                )

                # Softmax loss
                self.cost = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(
                        labels=self.get_y_placeholder(),
                        logits=pred
                    )
                ) + l2

                # Adam Optimizer
                self.optimizer = tf.train.AdamOptimizer(
                    learning_rate=self.hyperparams['learning_rate']
                ).minimize(self.cost)

                self.correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(self.get_tensor_by_name(Y_NAME), 1))
                self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))

                # To keep track of training's performance
                self.test_losses = []
                self.test_accuracies = []
                self.train_losses = []
                self.train_accuracies = []

                self.create_session()

                self.is_initialized = True

        return self

    def create_graph(self):
        self.graph = tf.Graph()

    def create_session(self):
        self.sess = tf.Session(config=tf.ConfigProto(log_device_placement=True), graph=self.graph)
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def get_tensor_by_name(self, name):
        return self.graph.get_tensor_by_name("{0}/{1}:0".format(LSTM_RNN_VARIABLE_SCOPE, name))

    def get_graph(self):
        return self.graph

    def get_session(self):
        return self.sess

    def get_x_placeholder(self):
        return self.get_tensor_by_name(X_NAME)

    def get_y_placeholder(self):
        return self.get_tensor_by_name(Y_NAME)

    def teardown(self):
        if self.sess is not None:
            self.sess.close()
        self.is_initialized = False

    def fit(self, data_inputs, expected_outputs=None) -> 'BaseStep':
        if not isinstance(data_inputs, np.ndarray):
            data_inputs = np.array(data_inputs)

        if not isinstance(expected_outputs, np.ndarray):
            expected_outputs = np.array(expected_outputs)

        if expected_outputs.shape != (len(data_inputs), self.hyperparams['n_classes']):
            expected_outputs = np.reshape(expected_outputs, (len(data_inputs), self.hyperparams['n_classes']))

        with tf.variable_scope(LSTM_RNN_VARIABLE_SCOPE, reuse=tf.AUTO_REUSE):
            _, loss, acc = self.sess.run(
                [self.optimizer, self.cost, self.accuracy],
                feed_dict={
                    self.get_x_placeholder(): data_inputs,
                    self.get_y_placeholder(): expected_outputs
                }
            )

            self.train_losses.append(loss)
            self.train_accuracies.append(acc)

            print("Batch Loss = " + "{:.6f}".format(loss) + ", Accuracy = {}".format(acc))

        self.is_invalidated = True

        return self

    def transform(self, data_inputs):
        if not isinstance(data_inputs, np.ndarray):
            data_inputs = np.array(data_inputs)

        with tf.variable_scope(LSTM_RNN_VARIABLE_SCOPE, reuse=tf.AUTO_REUSE):
            outputs = self.sess.run(
                [self.get_tensor_by_name(PRED_NAME)],
                feed_dict={
                    self.get_x_placeholder(): data_inputs
                }
            )[0]
            return outputs

# Neuraxle Pipeline 

In [6]:
class HumanActivityRecognitionPipeline(MiniBatchSequentialPipeline):
    def __init__(self):
        MiniBatchSequentialPipeline.__init__(self, [
            OutputTransformerWrapper(OneHotEncoder(nb_columns=N_CLASSES, name='one_hot_encoded_label')),
            ClassificationRNNTensorFlowModel(),
            Joiner(batch_size=BATCH_SIZE)
        ])


# Train Pipeline 

In [7]:
training_data_count = len(X_train)
training_iters = training_data_count * 3

pipeline = HumanActivityRecognitionPipeline()

no_iter = int(math.floor(training_iters / BATCH_SIZE))
for _ in range(no_iter):
    pipeline, outputs = pipeline.fit_transform(X_train, y_train)

pipeline.save(ExecutionContext.create_from_root(pipeline, ExecutionMode.FIT, DEFAULT_CACHE_FOLDER))

pipeline.teardown()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will al

HumanActivityRecognitionPipeline
(
	HumanActivityRecognitionPipeline(
	name=HumanActivityRecognitionPipeline,
	hyperparameters=HyperparameterSamples()
)(
		[('OutputTransformerWrapper',
  OutputTransformerWrapper(
	wrapped=OneHotEncoder(
	name=one_hot_encoded_label,
	hyperparameters=HyperparameterSamples()
),
	hyperparameters=HyperparameterSamples()
)),
 ('ClassificationRNNTensorFlowModel',
  ClassificationRNNTensorFlowModel(
	name=ClassificationRNNTensorFlowModel,
	hyperparameters=HyperparameterSamples([('n_steps', 128),
                       ('n_inputs', 9),
                       ('n_hidden', 32),
                       ('n_classes', 6),
                       ('learning_rate', 0.0025),
                       ('lambda_loss_amount', 0.0015),
                       ('batch_size', 1500)])
)),
 ('Joiner', Joiner(
	name=Joiner,
	hyperparameters=HyperparameterSamples()
))]	
)
)

# Serve Rest Api

In [8]:
pipeline = HumanActivityRecognitionPipeline()

pipeline = pipeline.load(ExecutionContext.create_from_root(pipeline, ExecutionMode.FIT, DEFAULT_CACHE_FOLDER))

# pipeline, outputs = pipeline.fit_transform(X_train, y_train)  # we could train further more here for instance.

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device

INFO:tensorflow:Restoring parameters from /home/gui/Documents/GIT/refactor_lstm/LSTM-Human-Activity-Recognition/cache/HumanActivityRecognitionPipeline/ClassificationRNNTensorFlowModel/ClassificationRNNTensorFlowModel.ckpt


In [ ]:
app = FlaskRestApiWrapper(
    json_decoder=CustomJSONDecoderFor2DArray(),
    wrapped=pipeline,
    json_encoder=CustomJSONEncoderOfOutputs()
).get_app()

app.run(debug=False, port=5000)

 * Serving Flask app "neuraxle.api.flask" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Nov/2019 22:38:52] "GET / HTTP/1.1" 200 -
